# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем таблицу и изучим общую информацию о данных

**Импортируем необходимые библиотеки:**

In [1]:
import pandas as pd

**Считаем данные из csv-файла в датафрейм `'data'`:**

In [2]:
data = pd.read_csv('/datasets/data.csv')

**Выведем первые 10 строк датафрейма `data` на экран:**

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Исследуем основную информацию о датафрейме с помощью метода `info()` и `describe()`:**

In [4]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Полученные данные содержат 12 столбцов и 21525 строк;**

**Столбцы `'days_employed'` и `'total_income'` содержат пропуски в данных;**

**Столбцы `'days_employed'` и `'children'` содержат аномалии в виде отрицательных значений;**

**Тип данных в столбце `'days_employed'` - вещественное число.**

## Шаг 2. Предобработка данных

**Найдём количество пропущенных значений в столбцах:**

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Пропущеные значения в столбце `'total_income'` заполним медианными значениями в зависимости от типа дохода в столбце `'income_type'`:**

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

**Заменим отрицательные значения в столбце `'days_employed'` положительными:**

In [7]:
data['days_employed'] = data['days_employed'].abs()

**Проверим медианное значение трудового стажа `'days_employed'` в днях для каждого типа занятости:**

In [8]:
data.groupby('income_type', as_index=False).agg({'days_employed':'median'})

,income_type,days_employed
0,безработный,366413.652744
1,в декрете,3296.759962
2,госслужащий,2689.368353
3,компаньон,1547.382223
4,пенсионер,365213.306266
5,предприниматель,520.848083
6,сотрудник,1574.202821
7,студент,578.751554


**У двух типов занятости (безработные и пенсионеры) получаются аномально большие значения.**

**Этот столбец не понадобится для исследования, пропуски можно заменить нулями:**

In [9]:
data['days_employed'] = data['days_employed'].fillna(0)

In [10]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Посмотрим на уникальные значения в столбце `'children'`:**

In [11]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**В столбце `'children'` есть аномальные значения (20 и -1).**

In [12]:
children = data[(data['children'] == -1) | (data['children'] == 20)].shape[0]
print(f"Аномальные значения в столбце 'children' составляют {round(children / data.shape[0] * 100, 2)}% от общего объёма данных.")

Аномальные значения в столбце 'children' составляют 0.57% от общего объёма данных.


**Данными значениями можно пренебречь поскольку они занимают менее 1% от общего объёма данных.**

In [13]:
data = data[(data['children'] != -1) & (data['children'] != 20)]
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

**Изменим вещественный тип данных в столбце `'total_income'` на целочисленный с помощью метода `astype()`:**

In [14]:
data['total_income'] = data['total_income'].astype(int)

**В столбце `'education'` есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру:**

In [15]:
data['education'] = data['education'].str.lower()
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Проверим данные на наличие дубликатов:**

In [16]:
duplicates = data.duplicated().sum()
print(f'Объём имеющихся данных составляет {data.shape[0]} строк;')
print(f'В данных присутствует {duplicates} дубликат;')
print(f'Дубликаты составляют {round(duplicates/data.shape[0]*100,2)}% от исходного объёма данных.')

Объём имеющихся данных составляет 21402 строк;
В данных присутствует 71 дубликат;
Дубликаты составляют 0.33% от исходного объёма данных.


**Дубликаты составляют незначительный объём исходных данных, поэтому их удаляем:**

In [17]:
data = data.drop_duplicates()
data.shape[0]

21331

**В зависимости от уровня доходов заёмщиков создадим в датафрейме столбец `'total_income_category'` со следующими категориями:**

- 0–30000 — `'< 30k'`;
- 30001–50000 — `'30-50k'`;
- 50001–200000 — `'50-200k'`;
- 200001–1000000 — `'200k-1M'`;
- 1000001 и выше — `'> 1M'`.

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return '< 30k'
        elif 30001 <= income <= 50000:
            return '30-50k'
        elif 50001 <= income <= 200000:
            return '50-200k'
        elif 200001 <= income <= 1000000:
            return '200k-1M'
        elif income >= 1000001:
            return '> 1M'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,200k-1M
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,50-200k
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,50-200k
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,200k-1M
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,50-200k


**Рассмотрим цели взятия кредита из столбца `'purpose'`:**

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Напишем функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,200k-1M,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,50-200k,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,50-200k,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,200k-1M,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,50-200k,проведение свадьбы


## Шаг 3.  Ответьте на вопросы:

**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

**Сгруппипуем общее число заёмщиков и должников по количеству детей:**

In [23]:
child_debt = data.groupby('children')['debt'].agg(['count', 'sum', 'mean']).reset_index()
child_debt['mean'] = round(child_debt['mean'] * 100, 2)
child_debt.columns = ['дети', 'число заемщиков', 'число должников', 'доля должников']
child_debt

,дети,число заемщиков,число должников,доля должников
0,0,14091,1063,7.54
1,1,4808,444,9.23
2,2,2052,194,9.45
3,3,330,27,8.18
4,4,41,4,9.76
5,5,9,0,0.00


**В полученной таблице представлены следующие данные:**

- В столбце `'дети'` заёмщики сгруппированы в зависимости от количества детей в семье;
- В столбце `'число заемщиков'` показано общее число заёмщиков; 
- В столбце `'число должников'` представлено число неплательщиков;
- Столбец `'доля должников'` отражает долю неплательщиков в зависимости от количества детей в семье.

Из представленных данных видно, что процент должников колеблется в диапазоне от 7,5% до 9,7%

Общее число заёмщиков, имеющих в семье 5 детей, ничтожно мало (0,04% от общего числа заёмщиков), поэтому считаю возможным этими данными пренебречь.

**Вывод: полученные результаты свидетельствуют об отсутствии явной зависимости между количеством детей и своевременным возвратом кредита, однако, похоже, что с возвратом денег плохо обстоят дела у семей с одним-двумя детьми, а лучше всего платят бездетные.** 

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [24]:
family_debt = data.pivot_table(index='family_status', values='debt',\
                               aggfunc=['count', 'sum', 'mean']).reset_index()

family_debt['mean'] = round(family_debt['mean'] * 100, 2)

family_debt.columns = ['семейное положение', 'число заемщиков',\
                       'число должников', 'доля должников']

family_debt = family_debt.sort_values(by='доля должников',\
                                      ascending=False).reset_index(drop=True)
family_debt

,семейное положение,число заемщиков,число должников,доля должников
0,Не женат / не замужем,2796,273,9.76
1,гражданский брак,4134,385,9.31
2,женат / замужем,12261,927,7.56
3,в разводе,1189,84,7.06
4,вдовец / вдова,951,63,6.62


**В полученной таблице представлены следующие данные:**

- В столбце `'семейное положение'` заёмщики сгруппированы по семейному положению;
- В столбце `'число заемщиков'` показано общее число заёмщиков; 
- В столбце `'число должников'` представлено число неплательщиков;
- Столбец `'доля должников'` отражает долю неплательщиков в зависимости от их семейного положения.

**Разница между различными категориями неплательщиков в зависимости от их семейного положения может составлять до 3%.
Вдовцы возвращают займы чуть лучше, чем заёмщики не состоящие в браке либо состоящие в гражданском браке.**

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [25]:
payback = data.groupby('total_income_category')['debt'].agg(['count', 'sum', 'mean']).reset_index()
payback['mean'] = round(payback['mean'] * 100, 2)
payback = payback.sort_values('mean', ascending=False).reset_index(drop=True)
payback.columns = ['уровень доходов', 'число заемщиков',\
                       'число должников', 'доля должников']
payback

,уровень доходов,число заемщиков,число должников,доля должников
0,< 30k,22,2,9.09
1,50-200k,15921,1353,8.50
2,> 1M,25,2,8.00
3,200k-1M,5014,354,7.06
4,30-50k,349,21,6.02


**В полученной таблице представлены следующие данные:**

- В столбце `'уровень доходов'` заёмщики сгруппированы в зависимости от дохода;
- В столбце `'число заемщиков'` показано общее число заёмщиков;
- В столбце `'число должников'` представлено число неплательщиков;
- Столбец `'доля должников'` отражает долю неплательщиков в зависимости от их доходов.

**Вывод: разница между различными категориями заёмщиков в зависимости от их дохода как и в предыдущих случаях не позволяет однозначно отнести какую-либо группу в разряд неблагонадёжных. Данные по наиболее и наименее обеспеченным категориям заёмщиков нельзя считать репрезентативными.**

**Люди с доходом ниже среднего оказались более добросовестными заёмщиками, чем люди со средним уровнем достатка.** 

**Как разные цели кредита влияют на его возврат в срок?**

In [26]:
loan_purpose = data.groupby('purpose_category')['debt'].agg(['count', 'sum', 'mean']).reset_index()
loan_purpose['mean'] = round(loan_purpose['mean'] * 100, 2)
loan_purpose = loan_purpose.sort_values(by='mean',\
                                      ascending=False).reset_index(drop=True)
loan_purpose.columns = ['цель кредита', 'число заемщиков',\
                       'число должников', 'доля должников']
loan_purpose

,цель кредита,число заемщиков,число должников,доля должников
0,операции с автомобилем,4279,400,9.35
1,получение образования,3988,369,9.25
2,проведение свадьбы,2313,183,7.91
3,операции с недвижимостью,10751,780,7.26


**В полученной таблице представлены следующие данные:**

- В столбце `'цель кредита'` представлены основные цели;
- В столбце `'число заемщиков'` показано общее число заёмщиков;
- В столбце `'число должников'` представлено число неплательщиков;
- Столбец `'доля должников'` отражает долю неплательщиков в зависимости от их доходов.

**Вывод: независимо от того, на какие цели был взят кредит, процент невыплаты сопоставим с предыдущими случаями, но можно выделить следующие тенденции: люди, решившиеся на ответственный шаг в своей жизни (например, связать себя узами брака или же обзавестись собственной крышей над головой) оказались более ответственными заёмщиками в сравнении со студентами (независимо от их возраста и социального статуса) и автомобилистами.**

## Общий вывод

**Ни один из предложенных вариантов оценки платёжеспособности заёмщиков не позволяет однозначно разделить их на благонадёжных и неблагонадёжных. Необходим более комплексный подход, позволяющий проводить оценку платёжеспособности на основе комбинации различных метрик.**

**На основе полученных данных можно предположить, что:**

**- Наименее надёжными заёмщиками являются люди не состоящие в браке, либо живущие в гражданском браке, имеющие одного ребёнка, средний уровень доходов, желающие повысить уровень своего образования либо приобрести автомобиль;**

**- Наибольшего доверия заслуживают граждане состоящие в настоящий момент (либо ранее состоявшие) в браке, не имеющие детей, с достатком чуть ниже среднего, желающие обзавестись собственным жильём, либо решившие (снова?) связать себя узами брака.**